In [2]:
import Pkg
Pkg.add("JLD2")
Pkg.add("Flux")
Pkg.add("Optimisers")
Pkg.add("Printf")
Pkg.add("Statistics")


    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed JLD2 ─ v0.5.13
    Updating `~/.julia/environments/v1.10/Project.toml`
  [033835bb] + JLD2 v0.5.13
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [033835bb] + JLD2 v0.5.13
Precompiling packages...
  59520.7 ms  ✓ JLD2
  1 dependency successfully precompiled in 69 seconds. 462 already precompiled.
   Resolving package versions...
   Installed RealDot ───────────── v0.1.0
   Installed PrettyPrint ───────── v0.2.0
   Installed ContextVariablesX ─── v0.1.3
   Installed ZygoteRules ───────── v0.2.7
   Installed ShowCases ─────────── v0.1.0
   Installed InitialValues ─────── v0.3.1
   Installed Accessors ─────────── v0.1.42
   Installed MLUtils ───────────── v0.4.8
   Installed MLCore ────────────── v1.0.0
   Installed FLoopsBase ────────── v0.1.1
   Installed DefineSingletons ──── v0.1.2
   Installed MicroCollections ──── v0.2.0
   Installed ProgressLogging ───── v0.1.4
   I

In [1]:
using JLD2
using Flux
using Printf
using Statistics
using Optimisers

In [2]:
X_train = load("/content/imdb_dataset_prepared.jld2", "X_train")
y_train = load("/content/imdb_dataset_prepared.jld2", "y_train")
X_test = load("/content/imdb_dataset_prepared.jld2", "X_test")
y_test = load("/content/imdb_dataset_prepared.jld2", "y_test")
embeddings = load("/content/imdb_dataset_prepared.jld2", "embeddings")
vocab = load("/content/imdb_dataset_prepared.jld2", "vocab")
nothing

embedding_dim = size(embeddings,1)

50

In [3]:
model = Chain(
    Flux.Embedding(length(vocab), embedding_dim),
    x->permutedims(x, (2,1,3)),
    Conv((3,), embedding_dim => 8, relu),
    MaxPool((8,)),
    Flux.flatten,
    Dense(128, 1, σ)
)

Chain(
  Embedding(12849 => 50),               # 642_450 parameters
  var"#1#2"(),
  Conv((3,), 50 => 8, relu),            # 1_208 parameters
  MaxPool((8,)),
  Flux.flatten,
  Dense(128 => 1, σ),                   # 129 parameters
)                   # Total: 5 arrays, 643_787 parameters, 2.456 MiB.

In [4]:
# add Glove embeddings to Embedding layer
model.layers[1].weight .= embeddings;

In [5]:
dataset = Flux.DataLoader((X_train, y_train), batchsize=64, shuffle=true)
loss(m, x, y) = Flux.Losses.binarycrossentropy(m(x), y)
accuracy(m, x, y) = mean((m(x) .> 0.5) .== (y .> 0.5))
opt = Optimisers.setup(Adam(), model)
epochs = 5

for epoch in 1:epochs
    println("\nEpoch: $epoch")

    total_loss = 0.0
    total_acc = 0.0
    num_samples = 0
    total_batch_time = 0.0
    total_batch_alloc = 0
    total_batch_gc_time = 0.0

    for (x, y) in dataset
        # Pomiar statystyk batcha (podobnie jak @timed)
        stats = @timed begin
            grads = Flux.gradient(model) do m
                loss(m, x, y)
            end
            Optimisers.update!(opt, model, grads[1])
        end

        # Akumulacja loss i accuracy
        total_loss += loss(model, x, y)
        total_acc += accuracy(model, x, y)
        num_samples += 1

        # Akumulacja statystyk podobnie jak w twoim drugim kodzie
        total_batch_time += stats.time
        total_batch_alloc += stats.bytes
        total_batch_gc_time += stats.gctime
    end

    # Obliczenie średnich wartości
    train_loss = total_loss / num_samples
    train_acc = total_acc / num_samples

    # Obliczenie test metrics
    test_acc = accuracy(model, X_test, y_test)
    test_loss = loss(model, X_test, y_test)

    # Wyświetlenie w formacie podobnym do drugiego kodu
    println(@sprintf("Epoch: %d \tTrain: (l: %.4f) \tTotal Epoch Time: %.4fs \tTotal Alloc: %s \tGC Time: %.4fs",
            epoch, train_loss, total_batch_time, Base.format_bytes(total_batch_alloc), total_batch_gc_time))
end


Epoch: 1
Epoch: 1 	Train: (l: 0.5721) 	Total Epoch Time: 52.3239s 	Total Alloc: 8.289 GiB 	GC Time: 1.7787s

Epoch: 2
Epoch: 2 	Train: (l: 0.3357) 	Total Epoch Time: 12.8267s 	Total Alloc: 6.142 GiB 	GC Time: 0.6593s

Epoch: 3
Epoch: 3 	Train: (l: 0.2488) 	Total Epoch Time: 13.3807s 	Total Alloc: 6.142 GiB 	GC Time: 0.2530s

Epoch: 4
Epoch: 4 	Train: (l: 0.1905) 	Total Epoch Time: 13.8692s 	Total Alloc: 6.142 GiB 	GC Time: 0.2631s

Epoch: 5
Epoch: 5 	Train: (l: 0.1400) 	Total Epoch Time: 13.8960s 	Total Alloc: 6.142 GiB 	GC Time: 0.2660s


In [6]:
# Funkcje loss i accuracy (te same co w treningu)
loss(m, x, y) = Flux.Losses.binarycrossentropy(m(x), y)
accuracy(m, x, y) = mean((m(x) .> 0.5) .== (y .> 0.5))

# Rozpoczęcie pomiaru czasu testowania
test_time = @elapsed begin
    # Obliczenie test loss i accuracy
    test_loss_value = loss(model, X_test, y_test)
    test_accuracy_value = accuracy(model, X_test, y_test)
end

# Konwersja accuracy na procenty
test_accuracy_percent = test_accuracy_value * 100

# Wyświetlenie wyników w formacie PyTorch
println(@sprintf("Test Loss (czas: %.2fs): %.4f", test_time, test_loss_value))
println(@sprintf("Test Accuracy: %s %%", test_accuracy_percent))

Test Loss (czas: 4.16s): 0.3249
Test Accuracy: 87.53 %
